In [7]:
#New file to vectorise the reviews to perform the data science on them

In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math


<b><h2>Handling of the review data and vectorising of the text

In [2]:
full_data = pd.read_csv("../data/review_dataset.csv")
full_data = pd.DataFrame(full_data)
full_data.dropna()
full_data.drop("Unnamed: 0", axis=1)

,Business Name,Street Number,Route,Postal Town,County,Country,Postcode,Overall Rating,Language,Rating,Review Description,Time,Date,processed review,lemmatized review,Keyword Contains,Keyword No Negative
0,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,have to say this is the best salon ive ever be...,1.652418e+09,2022-05-13 05:07:20,say best salon ive ever 4 weeks lifting acryli...,say best salon ive ever 4 week lifting acryli...,False,False
1,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,nian is incredible ive been coming here for a ...,1.661441e+09,2022-08-25 15:22:45,nian incredible ive coming couple months every...,nian incredible ive coming couple month every...,False,False
2,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,i got a new set of gels from tee who was so so...,1.658412e+09,2022-07-21 14:04:52,got new set gels tee lovely chat helped choose...,got new set gel tee lovely chat helped choose...,False,False
3,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,i have been coming to this salon since i moved...,1.660734e+09,2022-08-17 10:57:00,coming salon since moved birmingham took find ...,coming salon since moved birmingham took find...,False,False
4,Nails and the City,46,Kenyon Street,Birmingham,West Midlands,United Kingdom,B18 6AR,4.7,en,5.0,i had my nails done for the first time and i c...,1.659646e+09,2022-08-04 20:41:01,nails done first time say place super cozy gir...,nail done first time say place super cozy gir...,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22374,Super Hand Car Wash,53-55,Llangollen Road,Wrexham,Wrexham Principal Area,United Kingdom,LL14 3RS,5.0,en,5.0,just had ford ranger polished so pleased with ...,1.654364e+09,2022-06-04 17:31:51,ford ranger polished pleased results vehicle l...,ford ranger polished pleased result vehicle l...,False,False
22375,Super Hand Car Wash,53-55,Llangollen Road,Wrexham,Wrexham Principal Area,United Kingdom,LL14 3RS,5.0,en,5.0,the name says it all fantastic service great p...,1.641225e+09,2022-01-03 15:44:22,name says fantastic service great prices top q...,name say fantastic service great price top qu...,False,False
22376,Super Hand Car Wash,53-55,Llangollen Road,Wrexham,Wrexham Principal Area,United Kingdom,LL14 3RS,5.0,en,5.0,a brilliant super hand car wash drove away spa...,1.652984e+09,2022-05-19 18:06:27,brilliant super hand car wash drove away spark...,brilliant super hand car wash drove away spar...,False,False
22377,Super Hand Car Wash,53-55,Llangollen Road,Wrexham,Wrexham Principal Area,United Kingdom,LL14 3RS,5.0,en,5.0,always happy when my black and yellow fiesta c...,1.654781e+09,2022-06-09 13:28:00,always happy black yellow fiesta comes gleamin...,always happy black yellow fiesta come gleamin...,False,False


In [5]:
from sklearn.model_selection import train_test_split


In [7]:
X_train_text, X_test_text = train_test_split(full_data,test_size=0.33)
y = X_train_text["Keyword Contains"].copy()
test_y = X_test_text["Keyword Contains"].copy()
X_train_text.drop("Keyword Contains",axis=1)
X_test_text.drop("Keyword Contains",axis=1)

,Unnamed: 0,Business Name,Street Number,Route,Postal Town,County,Country,Postcode,Overall Rating,Language,Rating,Review Description,Time,Date,processed review,lemmatized review,Keyword No Negative
197,197,Donnas Nails & Beauty,196,Rosemount Place,Aberdeen,Aberdeen City,United Kingdom,AB25 3SS,4.9,en,5.0,great staff book in advance,1.563893e+09,2019-07-23 14:51:36,great staff book advance,great staff book advance,False
4688,4688,Natures Way Health & Beauty Centre,187,Tulketh Brow,Preston,Lancashire,United Kingdom,PR2 2JD,5.0,en,5.0,instant results from the lovely karen for a wr...,1.662054e+09,2022-09-01 17:43:41,instant results lovely karen wrinklereducing t...,instant result lovely karen wrinklereducing t...,False
17399,17399,London Road Hand Car Wash,1087,London Road,Leigh-on-Sea,Southend-on-Sea,United Kingdom,SS9 3JP,4.5,en,5.0,lovely people always happy good price recommend,1.590585e+09,2020-05-27 13:03:40,lovely people always happy good price recommend,lovely people always happy good price recommend,False
11049,11049,Bradleys Estate Agents Plymouth,16,Mannamead Road,Plymouth,Plymouth,United Kingdom,PL4 7AA,4.5,en,5.0,i cant rate bradleys high enough our family ha...,1.653582e+09,2022-05-26 16:12:18,cant rate bradleys high enough family used bra...,cant rate bradley high enough family used bra...,False
16163,16163,Rainbow Hand Car Wash,351,Elder Gate,Milton Keynes,Milton Keynes,United Kingdom,MK13 8HR,2.4,en,4.0,not bad but can do some improvements,1.605266e+09,2020-11-13 11:14:39,bad improvements,bad improvement,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9925,9925,Atlas Estate Agents,2,Allerton Road,Liverpool,Merseyside,United Kingdom,L18 1LN,4.8,en,5.0,this is my first time renting a house and atla...,1.653478e+09,2022-05-25 11:22:19,first time renting house atlas estate agent pe...,first time renting house atlas estate agent p...,True
263,263,Bangor nails,242,High Street,Bangor,Gwynedd,United Kingdom,LL57 1PA,4.9,en,5.0,love my nails so much staff are so nice aswell...,1.659213e+09,2022-07-30 20:24:36,love nails much staff nice aswell would defini...,love nail much staff nice aswell would defini...,False
11020,11020,Connells Estate Agents,Unit 6,Staniland Way,Peterborough,Peterborough,United Kingdom,PE4 6NA,4.8,en,5.0,a jewel in the crown of estate agents jason an...,1.652180e+09,2022-05-10 10:52:06,jewel crown estate agents jason team excellent...,jewel crown estate agent jason team excellent...,False
13312,13312,Y Homes Estate Agents,25,Hunters Way,York,York,United Kingdom,YO24 1JL,5.0,en,5.0,y homes handled our house purchase here in yor...,1.648638e+09,2022-03-30 11:00:10,homes handled house purchase york end summer 2...,home handled house purchase york end summer 2...,False


In [ ]:
count = 0
for i in X_train_text["Keyword Contains"]:
    if i == True:
        count += 1

print((count/len(x_train["Review Description"]))*100)

count = 0
for i in X_test_text["Keyword Contains"]:
    if i == True:
        count += 1

print((count/len(x_test["Review Description"]))*100)

<h3><b>Sklearn.feature_extraction attempt to vectorise the data



In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD

In [9]:
vectoriser = TfidfVectorizer(ngram_range=(1,5))
train_tfidf = vectoriser.fit_transform(X_train_text["lemmatized review"])
test_tfidf = vectoriser.transform(X_test_text["lemmatized review"])

# full_text = vectoriser.fit_transform(full_data['lemmatized review'])
# features = vectoriser.get_feature_names_out()

In [10]:
print("Train Shape: {} Rows and {} Columns".format(*train_tfidf.shape))
print("Test Shape: {} Rows and {} Columns".format(*test_tfidf.shape))

Train Shape: 14993 Rows and 1375671 Columns
Test Shape: 7386 Rows and 1375671 Columns


In [11]:
svd = TruncatedSVD(n_components=50, n_iter=20, random_state=42)
svd.fit(train_tfidf)

TruncatedSVD(n_components=50, n_iter=20, random_state=42)

In [12]:
X = svd.transform(train_tfidf)
test_tfidf = svd.transform(test_tfidf)

In [13]:
model = LogisticRegression()
model.fit(X,y)

LogisticRegression()

In [14]:
submission = model.predict(test_tfidf)
submission_df = pd.Series(submission).rename("Predicted Exploitation")
